In [3]:
# Triplet Select Samples
# Anchor Positive Negative
from nltk.stem import WordNetLemmatizer 
import nltk
import string
nltk.download('wordnet')


remove_words = ["@handle", "RT", "http"]
lemmatizer = WordNetLemmatizer() 
vocab = set()

def pre_process(sentence, max_length):
    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentence = sentence.split()
    target_remove = set()
    for i, token in enumerate(sentence):
        for target in remove_words:
            if (target == "http") and (target in token.lower()):
                target_remove.add(token)
                break
            if target in token:
                target_remove.add(token)
                break
    for target in target_remove:
        while target in sentence:
            sentence.remove(target)
            
    for i, token in enumerate(sentence):
        if len(token) == 0:
            continue
#         sentence[i] = token
#         print(sentence[i])
        vocab.add(sentence[i])

    max_length = max(max_length, len(sentence))
    sentence = ' '.join(sentence)
    return sentence, max_length

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hanxunhuang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
import csv
import collections
import numpy as np

train_file_path = "/Users/hanxunhuang/Desktop/UniMelb/2019-semester2/COMP90051-SML/project1/COMP90051-Project1/data/lemon_bear_files/lemonBearData/train_set_v1.txt"
train_dict = collections.defaultdict(list)
max_length = 0

length_array = []
with open(train_file_path, encoding='utf-8') as tsvfile:
    reader = tsvfile.readlines()
    for i, row in enumerate(reader):
        row = row.strip().split("\t")
        id = int(row[0])
        instance, max_length = pre_process(row[1], max_length)
        if not instance == "":
            train_dict[id].append(instance)
            length_array.append(len(instance.split()))
            if len(instance) == 0:
                print("Error")
    print("Total rows: %d" % i)
    
vocab = list(vocab)
print("Total ids: %d" % len(train_dict))
print("Longest Sentence: %d" % (max_length))
print("Vocab Size: %d" % (len(vocab)))

print(vocab[0], vocab[1])
    

Total rows: 291861
Total ids: 9266
Longest Sentence: 36
Vocab Size: 162591
gruden divergent


In [19]:
from pytorch_transformers import BertTokenizer, BertModel
import pickle

# with open("data/lemon_bear_files/lemonBearData/vocab.pickle", 'wb') as handle:
#     pickle.dump(vocab[:1000], handle, protocol=pickle.HIGHEST_PROTOCOL)

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer.add_tokens(vocab)
model.resize_token_embeddings(len(tokenizer))
print(len(tokenizer))

model.save_pretrained('checkpoints/v10/')
tokenizer.save_pretrained('checkpoints/v10/')

model = BertModel.from_pretrained('checkpoints/v10/', output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained('checkpoints/v10/')
print(len(tokenizer))

172484
172484


In [ ]:
# For Now, Just Use Random
import random
import pandas as pd
from tqdm import tnrange

df = pd.DataFrame()
df["anchor"] = ""
df["positive"] = ""
df["negative"] = ""

total_nums_of_rows = 1200000
ids = list(train_dict.keys())

for i in range(total_nums_of_rows):
    positive_id = random.choice(ids)
    negative_id = random.choice(ids)
    positive_sentence = train_dict[positive_id]
    negative_sentence = train_dict[negative_id]
    
    while len(positive_sentence) < 2 or positive_id == negative_id or len(negative_sentence) < 1:
        positive_id = random.choice(ids)
        negative_id = random.choice(ids)
        positive_sentence = train_dict[positive_id]
        negative_sentence = train_dict[negative_id]
        
    anchor = random.choice(positive_sentence)
    positive_sentence.remove(anchor)
    
    positive = random.choice(positive_sentence)
    positive_sentence.remove(positive)
    
    negative = random.choice(negative_sentence)
    negative_sentence.remove(negative)
    
    new_df = pd.DataFrame({"anchor": [anchor],
                           "positive": [positive],
                           "negative": [negative]})
    
    df = df.append(new_df, ignore_index = True)

    
    
    

In [ ]:
print(len(df))

In [ ]:
# Save to File
file_path = 'data/triplet_random/dataset.csv'
dataframe.to_csv(file_path, index=False, sep='\t', header=None)